In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import umap
import hdbscan

d:\Apps\anaconda3\envs\poop\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Wczytanie danych

In [2]:
hc_training = pd.read_csv('../../../classifier_data/HC_training.csv', sep='\t').T
hc_test = pd.read_csv('../../../classifier_data/HC_test.csv', sep='\t').T
oc_training = pd.read_csv('../../../classifier_data/OC_training.csv', sep='\t').T
oc_test = pd.read_csv('../../../classifier_data/OC_test.csv', sep='\t').T

synthetic_hc_data_path = '../../../best_models/HC/WGAN/500/synthetic_data/'
model_id = os.listdir(synthetic_hc_data_path)[0]
hc_synthetic = pd.read_csv(f'{synthetic_hc_data_path}/{model_id}/generated_data.tsv', sep='\t')

synthetic_oc_data_path = '../../../best_models/OC/WGAN/500/synthetic_data/'
model_id = os.listdir(synthetic_oc_data_path)[0]
oc_synthetic = pd.read_csv(f'{synthetic_oc_data_path}/{model_id}/generated_data.tsv', sep='\t')

# combine training and test data with labels
hc_training['label'] = 'HC'
hc_test['label'] = 'HC'
oc_training['label'] = 'OC'
oc_test['label'] = 'OC'

hc_synthetic['label'] = 'HC'
oc_synthetic['label'] = 'OC'

training_data = pd.concat([hc_training, oc_training])
test_data = pd.concat([hc_test, oc_test])

In [3]:
# keep only the columns that are in both datasets
# Znalezienie wspólnych kolumn
common_columns = training_data.columns.intersection(test_data.columns)
common_columns = common_columns.intersection(hc_synthetic.columns)
common_columns = common_columns.intersection(oc_synthetic.columns)

# Uporządkowanie kolumn w real_data
training_data = training_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
test_data = test_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
hc_synthetic = hc_synthetic[common_columns]
oc_synthetic = oc_synthetic[common_columns]

common_columns

Index(['ENSG00000000938', 'ENSG00000005844', 'ENSG00000009790',
       'ENSG00000010327', 'ENSG00000011422', 'ENSG00000011600',
       'ENSG00000012779', 'ENSG00000013364', 'ENSG00000013441',
       'ENSG00000018280',
       ...
       'ENSG00000229164', 'ENSG00000231500', 'ENSG00000240356',
       'ENSG00000244038', 'ENSG00000244734', 'ENSG00000257207',
       'ENSG00000266714', 'ENSG00000269028', 'ENSG00000271043', 'label'],
      dtype='object', length=356)

In [4]:
# add synthetic data to training data
training_data_synthetic = pd.concat([training_data, hc_synthetic, oc_synthetic])
training_data_synthetic

,ENSG00000000938,ENSG00000005844,ENSG00000009790,ENSG00000010327,ENSG00000011422,ENSG00000011600,ENSG00000012779,ENSG00000013364,ENSG00000013441,ENSG00000018280,...,ENSG00000229164,ENSG00000231500,ENSG00000240356,ENSG00000244038,ENSG00000244734,ENSG00000257207,ENSG00000266714,ENSG00000269028,ENSG00000271043,label
VUMC-HC-0033-TR2591,4.432779,4.211676,4.252125,4.399111,3.083092,4.586923,4.211676,4.820505,4.328301,4.029213,...,3.920710,6.828857,3.535280,4.954165,6.995484,9.847484,4.465409,7.757440,6.525607,HC
Vumc-HD-70-TR1062,5.786592,5.247890,5.983595,3.522177,3.522177,6.188222,5.368589,4.842162,5.114879,3.837909,...,6.106566,9.405474,3.701701,5.908122,10.021953,8.262227,4.139052,6.645036,5.397116,HC
VUMC-HC0053-DOT-HD-48h-TR3087,5.526849,4.934939,5.442502,4.588670,3.535847,4.956248,4.559495,5.442502,4.467036,4.079708,...,5.442502,7.399434,7.783065,4.822471,7.050183,8.831693,4.030369,4.253528,3.720811,HC
Vumc-HD-149-TR932,4.923042,3.937986,4.556107,3.083092,3.083092,5.010208,4.234439,4.988981,4.275620,3.083092,...,4.234439,8.609324,9.503594,4.804614,8.661739,3.647372,3.808559,8.517492,6.826289,HC
Vumc-HD-36-TR1165,6.484772,5.938859,7.417415,5.998264,3.083092,5.998264,5.339481,6.262440,6.564767,5.241598,...,6.309796,10.296126,5.671874,5.596133,9.877834,3.862107,6.355612,7.025844,5.339481,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,3.801777,3.565917,7.588131,3.705098,3.434579,4.684740,3.835699,4.953022,5.638238,4.042814,...,3.964185,8.638218,6.108396,4.197161,10.548830,5.158650,4.726916,4.489359,4.152618,OC
77,7.083744,5.024013,5.772552,4.875307,5.103038,5.603008,4.331097,7.165335,5.693348,5.080854,...,4.584392,7.406508,6.723532,5.367866,11.014709,6.089077,5.279609,8.921524,5.340836,OC
78,3.607180,4.005172,5.859495,3.370577,4.070570,3.766275,3.403420,3.755452,4.783412,3.699752,...,3.634888,6.883414,9.551842,5.185882,11.541342,9.528371,3.501860,6.211729,6.528543,OC
79,3.732708,3.594363,6.240908,3.434596,3.270937,5.513862,3.748862,4.874514,5.112660,3.452194,...,4.029493,8.719942,5.130271,4.194926,10.184455,6.395909,3.881705,4.375269,3.868807,OC


### Wykorzystanie klasyfikatorów

In [5]:
# klasyfikacja z uyciem svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


def svm_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Klasyfikacja przy użyciu SVM
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


svm_classification(training_data, test_data)
svm_classification(training_data_synthetic, test_data)

Dokładność: 0.9081081081081082
              precision    recall  f1-score   support

          HC       0.94      0.93      0.93       131
          OC       0.84      0.85      0.84        54

    accuracy                           0.91       185
   macro avg       0.89      0.89      0.89       185
weighted avg       0.91      0.91      0.91       185

[[122   9]
 [  8  46]]
Dokładność: 0.8594594594594595
              precision    recall  f1-score   support

          HC       0.90      0.90      0.90       131
          OC       0.76      0.76      0.76        54

    accuracy                           0.86       185
   macro avg       0.83      0.83      0.83       185
weighted avg       0.86      0.86      0.86       185

[[118  13]
 [ 13  41]]


In [6]:
from sklearn.tree import DecisionTreeClassifier
# klasyfikacja z uyciem random forest
def decision_tree_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    dt = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

decision_tree_classification(training_data, test_data)
decision_tree_classification(training_data_synthetic, test_data)

Dokładność: 0.7945945945945946
              precision    recall  f1-score   support

          HC       0.83      0.89      0.86       131
          OC       0.67      0.57      0.62        54

    accuracy                           0.79       185
   macro avg       0.75      0.73      0.74       185
weighted avg       0.79      0.79      0.79       185

[[116  15]
 [ 23  31]]


Dokładność: 0.8162162162162162
              precision    recall  f1-score   support

          HC       0.84      0.92      0.88       131
          OC       0.74      0.57      0.65        54

    accuracy                           0.82       185
   macro avg       0.79      0.75      0.76       185
weighted avg       0.81      0.82      0.81       185

[[120  11]
 [ 23  31]]


In [7]:
from sklearn.ensemble import RandomForestClassifier
# klasyfikacja z uyciem random forest
def random_forest_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

random_forest_classification(training_data, test_data)
random_forest_classification(training_data_synthetic, test_data)

Dokładność: 0.8486486486486486
              precision    recall  f1-score   support

          HC       0.83      0.98      0.90       131
          OC       0.93      0.52      0.67        54

    accuracy                           0.85       185
   macro avg       0.88      0.75      0.78       185
weighted avg       0.86      0.85      0.83       185

[[129   2]
 [ 26  28]]
Dokładność: 0.8540540540540541
              precision    recall  f1-score   support

          HC       0.84      0.98      0.91       131
          OC       0.94      0.54      0.68        54

    accuracy                           0.85       185
   macro avg       0.89      0.76      0.79       185
weighted avg       0.87      0.85      0.84       185

[[129   2]
 [ 25  29]]


In [8]:
from xgboost import XGBClassifier

# klasyfikacja z uyciem xgboost
def xgboost_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    y_train = y_train.replace('HC', 0)
    y_train = y_train.replace('OC', 1)
    y_test = y_test.replace('HC', 0)
    y_test = y_test.replace('OC', 1)
    xgb = XGBClassifier(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

xgboost_classification(training_data, test_data)
xgboost_classification(training_data_synthetic, test_data)

Dokładność: 0.8702702702702703
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       131
           1       0.83      0.70      0.76        54

    accuracy                           0.87       185
   macro avg       0.86      0.82      0.84       185
weighted avg       0.87      0.87      0.87       185

[[123   8]
 [ 16  38]]
Dokładność: 0.8594594594594595
              precision    recall  f1-score   support

           0       0.88      0.93      0.90       131
           1       0.80      0.69      0.74        54

    accuracy                           0.86       185
   macro avg       0.84      0.81      0.82       185
weighted avg       0.86      0.86      0.86       185

[[122   9]
 [ 17  37]]


In [9]:
from sklearn.neural_network import MLPClassifier

# klasyfikacja z uzyciem sieci neuronowych

def nn_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    nn = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42)
    nn.fit(X_train, y_train)
    y_pred = nn.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

nn_classification(training_data, test_data)
nn_classification(training_data_synthetic, test_data)

Dokładność: 0.918918918918919
              precision    recall  f1-score   support

          HC       0.93      0.96      0.94       131
          OC       0.90      0.81      0.85        54

    accuracy                           0.92       185
   macro avg       0.91      0.89      0.90       185
weighted avg       0.92      0.92      0.92       185

[[126   5]
 [ 10  44]]
Dokładność: 0.8918918918918919
              precision    recall  f1-score   support

          HC       0.91      0.94      0.92       131
          OC       0.84      0.78      0.81        54

    accuracy                           0.89       185
   macro avg       0.88      0.86      0.87       185
weighted avg       0.89      0.89      0.89       185

[[123   8]
 [ 12  42]]
